# Assignment 2 -- Text classification

by Kolokathi Fotini (DS3516007)   and   Spartalis Iosif (DS3516018)

In [32]:
import os
import re
import nltk
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA,TruncatedSVD
import random
from sklearn import metrics
import time


 
class LemmaTokenizer(object):
    """custom tokenizer with lemmatizer to be used in sklearn countVectorizer"""
    def __init__(self):
        self.tkzer = RegexpTokenizer(r'\b[^\d\W]+\b') #This tokenizer removes punctuation and numbers
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        filtered_lemm = []
        for w in self.tkzer.tokenize(doc):
            noun = self.wnl.lemmatize(w, pos='n')
            verb = self.wnl.lemmatize(w, pos='v')
            if(noun!=w):
                filtered_lemm.append(noun)
            else:
                filtered_lemm.append(verb)
            
        return filtered_lemm 

'''
input: a string which contains the path of  data and a 'start'  and an 'end' number of the folder name
i.e if we want to open from part1 to part9 folders then start=1 and end=10
output: one list in which each element is a text/email and another list in which each element is the target name of the 
corresponding text(in our case 'spam' or 'ham')
'''
def load_data(train_dir,start,end):
    "keep path of files of training data"
    emails_path = [os.path.join(train_dir+ str(i),f) for i in range(start,end) for f in os.listdir(train_dir + str(i))] 
    emails = []
    targets = []
    i=0
    for email_path in emails_path:
        with open(email_path,encoding='utf-8') as f:
            emails.append(f.readlines()[1:][1]) # we do not take the first line of each emails which is the subject
            file_name = emails_path[i].split("\\")[3]
            if(file_name[0:5]=="spmsg"):
                targets.append('spam')
            else:
                targets.append('ham')    
        i+=1
    return emails,targets

'''
unput : 
    tfs: doc-term numpy array in which each element contains the corresponding tf(t,d)
output:
    a tf-idf numpy array with normalized term vectors
'''
def tf_idf(tfs):
    idf_list = []
    # total number of docs
    N = tfs.shape[0] 
    # total number of terms
    terms = tfs.shape[1] 
    '''Here we calculate the idf for each term of doc-term matrix and append each of it in a list'''
    for i in range (0,terms):
        '''
        see: 
        http://stackoverflow.com/questions/36966019/how-aretf-idf-calculated-by-the-scikit-learn-tfidfvectorizer
        http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer    
        http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

        idf(d, t) = log [ (1 + N) / (1 + df(d, t)) ] + 1.
        '''
        idf_list.append((np.log((N+1)/(np.count_nonzero(tfs[:,i])+1))+1))

    '''Here we create a numpy array in which we have concatenated the idf_list N times'''
    idf_array = np.array(idf_list) # convert idf_list to numpy array
    idfs = np.tile(idf_array,(N,1))
    ''' tf-idf(d, t) = tf(d,t) * idf(d, t)'''
    tf_idf = tfs*idfs
    ''' Normalize term vectors of tf_idf array with l2 norm'''
    # see: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html#sklearn.preprocessing.normalize
    tf_idf_normalized = normalize(tf_idf,axis=0)
    return tf_idf_normalized

'''
input:
    train_X: tf-idf array of training data
    test_X: tf-idf array of test data
    k: number of most important principal components we will keep after dimentionality reduction
output:
    train_X and test_X transformed numpy arrays
'''
def select_features_pca(train_X, test_X, k=1000):
    selector = PCA(n_components=k)
    selector.fit(train_X)
    train_X = selector.transform(train_X)
    test_X = selector.transform(test_X)
    return train_X, test_X

'''
In practice TruncatedSVD is useful on large sparse datasets which cannot be centered without making the memory usage 
explode.
see:
http://scikit-learn.org/stable/modules/decomposition.html
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html

input:
    train_X: tf-idf array of training data
    test_X: tf-idf array of test data
    k: number of most important principal components we will keep after dimentionality reduction
output:
    train_X and test_X transformed numpy arrays
'''
def select_features_svd(train_X, test_X, k=1000):
    selector = TruncatedSVD(n_components=k)
    selector.fit(train_X)
    train_X = selector.transform(train_X)
    test_X = selector.transform(test_X)
    return train_X, test_X

'''
input:
   test_targets: list with the test targets 
   y_pred: list with the predicted test targets
'''
def report(test_targets,y_pred):
    '''prints classification report and confusion matrix'''
    target_names = ['spam','ham']
    print('Accuracy: %s' % metrics.accuracy_score(test_targets,y_pred))
    print()
    '''Classification report'''
    print('------------------------Classification report------------------------')
    print()
    print(classification_report(test_targets, y_pred, target_names=target_names))
    print('---------------------------------------------------------------------')
    print('Confusion matrix:')
    print(metrics.confusion_matrix(test_targets,y_pred))

## Load and preprocessing data

In [33]:
train_dir =".\\bare\\part"

'''load data'''
emails,targets = load_data(train_dir,1,11)

'''shuffle data'''
c = list(zip(emails, targets))
random.shuffle(c)
emails, targets = zip(*c)
emails = list(emails)
targets = list(targets)

'''split targets for training and test data'''
train_targets = targets[:int((len(targets)+1)*.80)] # 80% to training set
test_targets = targets[int(len(targets) * .80+1):] # 20% to test data


In [34]:

'''prepocessing of data'''
data= emails


* ##  Tf-idf 

In [35]:
'''document-term matrix with Tfs'''
vectorizer = CountVectorizer(min_df=1,tokenizer=LemmaTokenizer(), stop_words = 'english')
tfs = vectorizer.fit_transform(data)
tfs = tfs.toarray() # convert matrix to numpy array

'''tf-idf document-term numpy array'''
tf_idf_ = tf_idf(tfs)

In [36]:
tf_idf_.shape

(2893, 46900)

In [37]:
# Dimentionality reduction

'''SVD'''
train_data_svd,test_data_svd = select_features_svd(tf_idf_[:int((tf_idf_.shape[0]+1)*.80)],tf_idf_[int(tf_idf_.shape[0]*.80+1):])

### Execute Classification

* #### Logistic Regression

In [38]:
# Import Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


# http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

start_time = time.time()

# Here C=1/lambda.So larger C ==> weaker regularization(If we want to prevent underfitting)
# Smallar C ==> stronger regularization (If we want to prevent overfitting)
param_grid = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'fit_intercept':[True,False],'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag'],
             'tol':[1e-2,1e-3,1e-4,1e-5]}

# Hyperparameter cv=3  by default, so we make 3-fold cross validation.If we set cv=10 it takes more time to run
gs_clf = GridSearchCV(LogisticRegression(), param_grid,n_jobs=-1)

'''train classifier'''
gs_clf = gs_clf.fit(train_data_svd,train_targets)

elapsed_time = time.time() - start_time
print(elapsed_time)

127.5233404636383


In [39]:
'''Estimator values.It has default parameters that we did not specify'''
gs_clf.best_estimator_

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.01,
          verbose=0, warm_start=False)

In [40]:
'''Get the best parameter values directrly'''
gs_clf.best_params_

{'C': 10, 'fit_intercept': False, 'solver': 'newton-cg', 'tol': 0.01}

In [41]:
'''make prediction'''
y_pred = gs_clf.predict(test_data_svd)

'''Classification report'''
report(test_targets,y_pred)


Accuracy: 0.979238754325

------------------------Classification report------------------------

             precision    recall  f1-score   support

       spam       0.98      1.00      0.99       475
        ham       0.99      0.89      0.94       103

avg / total       0.98      0.98      0.98       578

---------------------------------------------------------------------
Confusion matrix:
[[474   1]
 [ 11  92]]


In [42]:
'''Using the best parameters to make predictions with logistic regression'''

''' Load classifier '''
LR = LogisticRegression(C=gs_clf.best_params_['C'],fit_intercept=gs_clf.best_params_['fit_intercept'],
                        solver=gs_clf.best_params_['solver'],tol=gs_clf.best_params_['tol'])

'''Train classifier''' 
regr_clf = LR.fit(train_data_svd,train_targets)

'''make prediction'''
Y_pred_regr = regr_clf.predict(test_data_svd)

'''Classification report''' 
report(test_targets,Y_pred_regr)


Accuracy: 0.979238754325

------------------------Classification report------------------------

             precision    recall  f1-score   support

       spam       0.98      1.00      0.99       475
        ham       0.99      0.89      0.94       103

avg / total       0.98      0.98      0.98       578

---------------------------------------------------------------------
Confusion matrix:
[[474   1]
 [ 11  92]]


* #### KNN

In [43]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import RandomizedSearchCV
from sklearn.metrics import classification_report

start_time = time.time()

# define the parameter values that should be searched
k_range = list(range(1, 31))

# Another parameter besides k that we might vary is the weights parameters
# uniform (all points in the neighborhood are weighted equally)
# distance (weights closer neighbors more heavily than further neighbors)

param_grid = {'n_neighbors':k_range,'weights':['uniform','distance'],'algorithm':['auto','ball_tree','kd_tree','brute'],
             'metric':['euclidean','cityblock','minkowski'],'p':[1,2]}

gs_clf = RandomizedSearchCV(KNeighborsClassifier(),param_grid,n_jobs=-1)

'''train classifier'''
gs_clf = gs_clf.fit(train_data_svd,train_targets)

elapsed_time = time.time() - start_time
print(elapsed_time)

14.62877893447876


In [44]:
'''Estimator values.It has default parameters that we did not specify'''
gs_clf.best_estimator_

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=24, p=2,
           weights='distance')

In [45]:
'''Get the best parameter values directrly'''
gs_clf.best_params_

{'algorithm': 'brute',
 'metric': 'euclidean',
 'n_neighbors': 24,
 'p': 2,
 'weights': 'distance'}

In [46]:
from sklearn import metrics
from sklearn.metrics import classification_report

'''make prediction'''
y_pred = gs_clf.predict(test_data_svd)

''' Classification report'''
report(test_targets,y_pred)


Accuracy: 0.970588235294

------------------------Classification report------------------------

             precision    recall  f1-score   support

       spam       0.99      0.98      0.98       475
        ham       0.91      0.93      0.92       103

avg / total       0.97      0.97      0.97       578

---------------------------------------------------------------------
Confusion matrix:
[[465  10]
 [  7  96]]


In [47]:
'''Using the best parameters to make predictions with KNN'''


''' Load classifier '''
KNN = KNeighborsClassifier(n_neighbors=gs_clf.best_params_['n_neighbors'],weights=gs_clf.best_params_['weights'],algorithm=gs_clf.best_params_['algorithm'],
                           metric=gs_clf.best_params_['metric'],p=gs_clf.best_params_['p'])

'''Train classifier''' 
knn_clf = KNN.fit(train_data_svd,train_targets)

'''make prediction'''
Y_pred_knn = knn_clf.predict(test_data_svd)

'''Classification report''' 
report(test_targets,Y_pred_knn)

Accuracy: 0.970588235294

------------------------Classification report------------------------

             precision    recall  f1-score   support

       spam       0.99      0.98      0.98       475
        ham       0.91      0.93      0.92       103

avg / total       0.97      0.97      0.97       578

---------------------------------------------------------------------
Confusion matrix:
[[465  10]
 [  7  96]]


* #### SVM

In [48]:
from sklearn.model_selection import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.svm import SVC
import scipy.stats

start_time = time.time()

# For hyperparameter C see:
# https://stats.stackexchange.com/questions/31066/what-is-the-influence-of-c-in-svms-with-linear-kernel

param_grid = {'C': scipy.stats.expon(scale=100),
              'gamma': scipy.stats.expon(scale=.1), 'kernel':['linear','poly','rbf','sigmoid']}


gs_clf = RandomizedSearchCV(SVC(), param_grid,n_jobs=-1)

'''train classifier'''
gs_clf = gs_clf.fit(train_data_svd,train_targets)

elapsed_time = time.time() - start_time
print(elapsed_time)


19.974853992462158


In [49]:
'''Estimator values.It has default parameters that we did not specify'''
gs_clf.best_estimator_

SVC(C=46.413398313918456, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.12751889220079624,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [50]:
'''Get the best parameter values directrly'''
gs_clf.best_params_

{'C': 46.413398313918456, 'gamma': 0.12751889220079624, 'kernel': 'rbf'}

In [51]:
from sklearn import metrics
from sklearn.metrics import classification_report

'''make prediction'''
y_pred = gs_clf.predict(test_data_svd)

''' Classification report'''
report(test_targets,y_pred)

Accuracy: 0.963667820069

------------------------Classification report------------------------

             precision    recall  f1-score   support

       spam       0.96      1.00      0.98       475
        ham       0.98      0.82      0.89       103

avg / total       0.96      0.96      0.96       578

---------------------------------------------------------------------
Confusion matrix:
[[473   2]
 [ 19  84]]


In [52]:
'''Using the best parameters to make predictions with SVM'''


''' Load classifier '''
SVM = SVC(C=gs_clf.best_params_['C'],gamma=gs_clf.best_params_['gamma'],kernel=gs_clf.best_params_['kernel'])

'''Train classifier''' 
svm_clf = SVM.fit(train_data_svd,train_targets)

'''make prediction'''
Y_pred_svm = svm_clf.predict(test_data_svd)

'''Classification report''' 
report(test_targets,Y_pred_svm)

Accuracy: 0.963667820069

------------------------Classification report------------------------

             precision    recall  f1-score   support

       spam       0.96      1.00      0.98       475
        ham       0.98      0.82      0.89       103

avg / total       0.96      0.96      0.96       578

---------------------------------------------------------------------
Confusion matrix:
[[473   2]
 [ 19  84]]


* #### Naive Bayes

In [53]:
# For MultinomialNB you can see: http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html


'''
There are 3 versions of naive bayes algorithm as we can see from here: http://scikit-learn.org/stable/modules/naive_bayes.html

1.GaussianNB which is not a good fit for document classification.
2.BernoulliNB which is good for document classification but it accepts inly boolean features.
3.MultinomialNB is good for document classification.It is suitable for classification with discrete features (e.g., word counts 
for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, 
fractional counts such as tf-idf also work well.

As SVD may return negative elements, we cannot use MultinomialNB.So we will take tf-idf numpy array without making dimentionality
reduction.Of course we do not wait as good results as the previous algorithms gave.
'''

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

param_grid = {'fit_prior':[True,False]}
gs_clf = GridSearchCV(MultinomialNB(), param_grid)

'''train classifier'''
gs_clf = gs_clf.fit(tf_idf_[:int((tf_idf_.shape[0]+1)*.80)],train_targets)


In [54]:
'''Estimator values.It has default parameters that we did not specify'''
gs_clf.best_estimator_

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [55]:
'''Get the best parameter values directrly'''
gs_clf.best_params_

{'fit_prior': True}

In [56]:
from sklearn import metrics
from sklearn.metrics import classification_report

'''make prediction'''
y_pred = gs_clf.predict(tf_idf_[int(tf_idf_.shape[0]*.80+1):])

''' Classification report'''
report(test_targets,y_pred)

Accuracy: 0.769896193772

------------------------Classification report------------------------

             precision    recall  f1-score   support

       spam       0.97      0.74      0.84       475
        ham       0.43      0.91      0.59       103

avg / total       0.88      0.77      0.80       578

---------------------------------------------------------------------
Confusion matrix:
[[351 124]
 [  9  94]]


In [57]:
'''Using the best parameters to make predictions with Naive Bayes'''


''' Load classifier '''
NaiveBayes = MultinomialNB(fit_prior=gs_clf.best_params_['fit_prior'])

'''Train classifier''' 
nb_clf = NaiveBayes.fit(tf_idf_[:int((tf_idf_.shape[0]+1)*.80)],train_targets)

'''make prediction'''
Y_pred_nb = nb_clf.predict(tf_idf_[int(tf_idf_.shape[0]*.80+1):])

'''Classification report''' 
report(test_targets,Y_pred_nb)

Accuracy: 0.769896193772

------------------------Classification report------------------------

             precision    recall  f1-score   support

       spam       0.97      0.74      0.84       475
        ham       0.43      0.91      0.59       103

avg / total       0.88      0.77      0.80       578

---------------------------------------------------------------------
Confusion matrix:
[[351 124]
 [  9  94]]


* ## Word Embeddings

In [58]:
from nltk.tokenize import word_tokenize

'''
output:
    model: a dictionary which has as key the word and as value the numpy array with one row the corresponding vector
'''
def load_embeddings(gloveFile):
    f = open(gloveFile,'r',encoding='utf8')
    model = {}
    for line in f:
        word = line.split(' ')[0]
        model[word] = np.array(line.split(' ')[1:],dtype='f')
    return model


In [59]:
embeddings = load_embeddings('glove.6B.300d.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.300d.txt'

In [ ]:
'''

#ARGEI POLY...............
start_time = time.time()

# data: processed data
T = np.zeros((len(data),300))

for doc_id, text in enumerate(data):
    word_sequences=word_tokenize(text)
    c =0 
    for word in word_sequences:
        if word in list(embeddings):
            c+=1
            T[doc_id,:]+=embeddings[word]
    if(c>0):
        T[doc_id,:] = T[doc_id,:]/c
            
       
elapsed_time = time.time() - start_time
print(elapsed_time)


'''